<a href="https://colab.research.google.com/github/SuphanutN/geo_spellchecker/blob/master/Thai_Location_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install PyDrive
!pip install pythainlp

     |████████████████████████████████| 13.1MB 312kB/s 
     |████████████████████████████████| 747kB 41.7MB/s 


In [2]:
import pandas as pd

import pythainlp

from pythainlp.corpus.common import thai_words
from pythainlp.tokenize import word_tokenize, DEFAULT_WORD_DICT_TRIE
from pythainlp.util import trie

from pythainlp import spell
from pythainlp.spell import NorvigSpellChecker

import pickle

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1rW9WWcYz0025dLHGFsh70K5QJ0SwsTB4"}) 
downloaded.GetContentFile('geocode.csv')
downloaded2 = drive.CreateFile({'id':"1d5NDfuaBlCAyEzMWhcvpjMSmIpr2Sz8y"}) 
downloaded2.GetContentFile('longdomap-poi-contributed-2020070921.csv')

geo_file = pd.read_csv('geocode.csv')
map_file = pd.read_csv('longdomap-poi-contributed-2020070921.csv')

In [13]:
province_list = geo_file['ProvinceTH'].unique().tolist()
ampor_list = geo_file['AmporTH'].unique().tolist()
tumbol_list = geo_file['TumbolTH'].unique().tolist()

In [4]:
map_file.head()

,id,name_th,name_en,address_th,address_en,latitude,longitude,telephone,website,lastupdate,poi_type,username
0,10000001,บริษัท เมตามีเดีย เทคโนโลยี จำกัด,"Metamedia Technology Co., Ltd.",98 อาคารสาทร สแควร์ ออฟฟิศ ทาวเวอร์ ชั้น 4 ห้อ...,"Rm. 407, 4F, Sathorn Square Office Tower, 98 N...",13.722642,100.529316,+6621081790,http://www.mm.co.th,2020-04-17 02:12:58,building,pattara
1,10000003,ไทยอินโฟนิตี้,Thai Infonity,แขวงทุ่งสองห้อง เขตหลักสี่ กรุงเทพมหานคร 10210,"Thung Song Hong, Lak Si, Bangkok, 10210",13.887970,100.575757,NaN,NaN,2020-01-08 22:28:11,NaN,pattara
2,10000007,ตลาดหลักทรัพย์แห่งประเทศไทย,The Stock Exchange of Thailand,ถนนรัชดาภิเษก แขวงคลองเตย เขตคลองเตย กรุงเทพม...,NaN,13.722620,100.559362,0-2229-2000 หรือ 0-2229-2222,http://www.set.or.th,2020-01-08 22:28:11,Shop / Other public location,pattara
3,10000008,ธนาบุตร,ธนาบุตร,บริษัท ธนาบุตร จำกัด,"Rama 1 Rd., Wang Mai, Patumwan, Bangkok, 10400",13.746797,100.528862,NaN,http://www.tanabutr.co.th/,2020-01-08 22:28:11,building,pattara
4,10000009,คลินิกแพทย์หญิงเฟื่องเพชร,คลินิกแพทย์หญิงเฟื่องเพชร,486/130 ซอยเพชรบุรี 16 แขวงถนนเพชรบุรี เขตราชเ...,"486/130 Soi Phetburi 16, Thanon Phet Buri, Rat...",13.752618,100.531037,+6622156159,NaN,2020-01-08 22:28:11,"hospital, clinic, healthcare office",pattara


In [25]:
test_text = "31/33 ถ.อำมาตย์ ต.ในเมือง เมือง ขอนแก่น 40000"

geo_stopword = ["ถนน","ตำบล","แขวง","อำเภอ","เขต","จังหวัด","ถ.","ต.","อ.","จ.","ประเทศ"]
province_remove_word = ["จ.","จังหวัด"]
ampor_remove_word = ["อ.","อำเภอ","เขต"]
tumbol_remove_word = ["ต.","ตำบล","แขวง"]

text_seg = test_text.split(" ")
len_text = len(text_seg)

if len(text_seg) >= 5:
  valid = [0,0,0,0]
  zip_code = text_seg[len_text-1]
  if (len(zip_code) == 5):
    valid[0] = 1

  province = text_seg[len_text-2]
  for word in province_remove_word:
    if word in province:
      province = province.replace(word, '')
  if province in province_list:
    valid[1] = 1

  ampor = text_seg[len_text-3]
  for word in ampor_remove_word:
    if word in ampor:
      ampor = ampor.replace(word, '')
  if ampor in ampor_list:
    valid[2] = 1

  tumbol = text_seg[len_text-4]
  for word in tumbol_remove_word:
    if word in tumbol:
      tumbol = tumbol.replace(word, '')
    print(tumbol)
  if tumbol in tumbol_list:
    valid[3] = 1

else:
  print("wrong input format")

print(text_seg)
print(len_text)
print(province)
print(len(zip_code))
print(valid)

ในเมือง
ในเมือง
ในเมือง
['31/33', 'ถ.อำมาตย์', 'ต.ในเมือง', 'เมือง', 'ขอนแก่น', '40000']
6
ขอนแก่น
5
[1, 1, 0, 1]


In [7]:
address_list = map_file['address_th'].tolist()